In [8]:
import json
import pandas as pd
from datetime import datetime

In [11]:
LIST_OF_CHANNEL_IDS = [
"UCsDmESjqNPukDmVnuneLrqw",
"UCY6Ij8zOds0WJEeqCLOnqOQ",
"UClN24S5-fc7LD0JJW0FdGWw",
"UC3iNdSV_RQU7DHTGpqEW96w",
"UC_52i6mlrzOZs_NEuSepzAg",
"UCr3cBLTYmIK9kY0F_OdFWFQ",
"UCV6g95OBbVtFmN9uiJzkFqQ",
"UCgCKYs56-LKEPGQ99DzqQOg",
"UCOwxx9VnEnlFKt5EB70KTzQ",
"UC2Ds30pkifFVD0CE08wF50g",
"UC7UGbBVrqLlq6CRxWWUmyKw",
"UCn1XB-jvmd9fXMzhiA6IR0w",
"UCFeqAfEuKm7lIg2ddQzh61A",
"UCAlxwHEm1bWWYoe5VCSQYEg",
"UCz7iJPVTBGX6DNO1RNI2Fcg",
"UCOAUz4d1NUNxSHh_SOJtdGQ",
"UCGGTAB19HlHEWPwwmxHsEKA",
"UCOsATJw-IZgqGT8MFrHjKGg",
"UCsKVP_4zQ877TEiH_Ih5yDQ",
"UCOinp9hALPUcZeoL3Kmaviw",
"UCGwu0nbY2wSkW8N-cghnLpA",
"UCu6v4AdYxVH5fhfq9mi5llA",
"UCj74rJ9Lgl3WTngq675wxKg",
"UCNIuvl7V8zACPpTmmNIqP2A",
"UCo8bcnLyZH8tBIH9V1mLgqQ",
"UCR0O-1cvuPNxDosvSDLpWHg",
"UCvUmwreRrbxeR1mbmojj8fg",
"UCUKi4zY5ETSqrKAjTBgjM-g",
"UCDQBZcjYKP1J1Nu-Y0_D37Q",
"UCQ9HvHH-KRYHI5ynj2kbLwQ",
"UCRfg0SWjIHm_h95e4V8X5og",
"UCWPB0WpnMIy-g7zncwIhvQg",
"UCC-slLJZ4p4HOznMUcFn_2g",
#====moar
"UCj6CFdE3LSddaoszxq604TQ",
"UCXrtqz4fqf-OUEVpp2A1W0w",
"UChUg_Sd5i74SFoZH2XvagwQ",
"UC2hm5rD_IrfYRMfq5YQudgA",
"UC_nEHeUEVNY5ZYLRWg8KoZQ",
"UCrlzgjrKUa3AlrfSZTBG2fg",
"UCupWnlhB3PMW1w0CLfLCfug",
"UCsW0LA-ThH18OCrT9pa2zfQ",
"UCG-O5k7KKNPABULS20_MAMA",
"UCepPGz8AVCbggMl3BvboaBA",
"UCP21EOiKAkCGjv6kqS6UsZw",
"UC9z7EZAbkphEMg0SP7rw44A",
"UCZft8Ol-dMtguwam7bWItnw",
"UCYaxncRvOBu-vtdhxf6qciA",
"UCte3qx7A5OWTPSnNjV4oheg",
"UCRei8TBpt4r0WPZ7MkiKmVg",
"UC6bXfhbiX1PsSi_-pdQeHjg",
"UC_7lgWbCA_HZQkaxmmQgNoA",
"UCQO_C1sfUCpUzzV-gxQ-lxA",
"UCHw-JjzU80GKrGJwI1Uf1OQ",
"UCDEaY2XwwfKomQAJgGfehWw",
"UCI-92EMS3ZJ11gz-S_oyaSA",
"UC7Mk9RmlKT_x8FAsMOJn1cg",
#====
"UCnsem444vdU1HhS0mb2wwTA",
"UCnbtlei4RJMHWSUq4LKn_SQ", 
"UC9i7x4EzTBlNvslu1wWEB8Q",
"UC1nM46_BMKq4vd4jmd6iMHA",
"UCMNxwlfY6P5Rm40QOp8Hqsg",
"UCgmublDj4T0zEP7ngRRGkmg"
]
print(len(LIST_OF_CHANNEL_IDS), "channels to analyze")

62 channels to analyze


So I get the list of videos that a channel has. I don't yet have the specifics (aka metadata) on the videos yet because this API does not give me the information. What I have to do is filter this information to get a list of videos I need to throw at a different API to get the metadata. 

In [12]:
df = pd.read_json (r'searchListResponse.json')
channelIdToListOfVids = {} #channelId: []vids
listOfVidIds = []
#initialize the dicts
for channelId in LIST_OF_CHANNEL_IDS:
    channelIdToListOfVids[channelId] = []

# #youtuber
# #0~9ish: items
# #items: 0~46ish
count= 0

for index, row in df.iterrows():  
    listOfLists = df.iat[index, row.index.get_loc(row.first_valid_index())]
    for indList in listOfLists:
        for item in indList['items']:
            channelId = item['snippet']['channelId']
            if 'videoId' in item['id'].keys():
                snippet=item['snippet']
                vidId = item['id']['videoId']
                publishTime = snippet['publishTime']
                vidTitle = snippet['title']
                vidDesc = snippet['description']

                vidInfo = {'vidId' : vidId,
                'publishTime': publishTime,
                'vidTitle': vidTitle}
                channelIdToListOfVids[channelId].append(vidInfo)
                listOfVidIds.append(vidId)
                count+=1

print(len(channelIdToListOfVids.keys()), " keys detected")
vidCont=0
for i in channelIdToListOfVids.keys():
    vidCont+=len(channelIdToListOfVids[i])
print(count, " videos detected")
channelIdToListOfVids
df

62  keys detected
5224  videos detected


,UCnsem444vdU1HhS0mb2wwTA,UCnbtlei4RJMHWSUq4LKn_SQ,UC9i7x4EzTBlNvslu1wWEB8Q,UC1nM46_BMKq4vd4jmd6iMHA,UCMNxwlfY6P5Rm40QOp8Hqsg,UCsDmESjqNPukDmVnuneLrqw,UCY6Ij8zOds0WJEeqCLOnqOQ,UClN24S5-fc7LD0JJW0FdGWw,UC3iNdSV_RQU7DHTGpqEW96w,UC_52i6mlrzOZs_NEuSepzAg,...,UCte3qx7A5OWTPSnNjV4oheg,UCRei8TBpt4r0WPZ7MkiKmVg,UC6bXfhbiX1PsSi_-pdQeHjg,UC_7lgWbCA_HZQkaxmmQgNoA,UCQO_C1sfUCpUzzV-gxQ-lxA,UCHw-JjzU80GKrGJwI1Uf1OQ,UCDEaY2XwwfKomQAJgGfehWw,UCI-92EMS3ZJ11gz-S_oyaSA,UC7Mk9RmlKT_x8FAsMOJn1cg,UCgmublDj4T0zEP7ngRRGkmg
0,"[{'kind': 'youtube#searchListResponse', 'etag'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,"[{'kind': 'youtube#searchListResponse', 'etag'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,"[{'kind': 'youtube#searchListResponse', 'etag'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,"[{'kind': 'youtube#searchListResponse', 'etag'...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,"[{'kind': 'youtube#searchListResponse', 'etag'...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"[{'kind': 'youtube#searchListResponse', 'etag'...",NaN,NaN,NaN
59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'kind': 'youtube#searchListResponse', 'etag'...",NaN,NaN
60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'kind': 'youtube#searchListResponse', 'etag'...",NaN
61,NaN,NaN,"[{'kind': 'youtube#searchListResponse', 'etag'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
with open('listOfVideoIds.json', 'w') as outfile:
    json.dump(listOfVidIds, outfile)

We now have the list of videoIds for each channel

After using this information in `main.py` to shoot at the API to get individual video metadata, I again filter out information specific to each video.

In [16]:
df = pd.read_json (r'videoListResponse.json')
backend_purposes= {}

def flattenList(list):
    flat_list = [item for sublist in list for item in sublist]
    return flat_list
    
for channelId in LIST_OF_CHANNEL_IDS:
    backend_purposes[channelId]=[]

matching_vidId_to_info={}
count = 0
debugMsg = []
setOfTypes = []
for index, indList in df.iterrows():
    items = indList['items']
    for item in items:
        count+=1
        vidId = item['id']
        channelId = item['snippet']['channelId']
        contentDetails= item['contentDetails']
        duration= contentDetails['duration']
        statistics = item['statistics']
        viewCount = statistics['viewCount']
        likeCount = statistics.get('likeCount')
        commentCount = statistics.get('commentCount')
        description = item['snippet']['description']
        if (likeCount == None or commentCount == None):
            print(vidId, "has", "likes" if likeCount == None else '', "comments" if commentCount == None else '', "disabled")
        topicCategories = []
        if item.get('topicDetails') != None:
            topicCategories = item.get('topicDetails').get('topicCategories')
            setOfTypes.append([i for i in topicCategories])
        if True not in ['music' in strng.lower() for strng in topicCategories] or item.get('topicDetails') == None: #all videos that are not music-typed or just don't have a type
            matching_vidId_to_info[vidId] = {'duration': duration,
                            'viewCount': int(-1 if viewCount is None else viewCount),
                            'likeCount': int(-1 if likeCount is None else likeCount),
                            'commentCount': int(-1 if commentCount is None else commentCount),
                            'topicCategories': topicCategories,
                            'description': description}
            backend_purposes[channelId].append(vidId)
        else: #all videos that are music-typed
            debugMsg.append(vidId)

vidIdsCollected =0 
for i in backend_purposes.keys():
    vidIdsCollected += len(backend_purposes[i])
print(vidIdsCollected, " videos to run semantic analysis on")
print(set(flattenList(setOfTypes)))
print("There were ", count-len(matching_vidId_to_info), " videos categorized as music-videos")
df.head()

QQilWUln3F8 has  comments disabled
0eCXTYobXQk has  comments disabled
hpQQohcHk9Q has  comments disabled
fV15ba-dpPI has  comments disabled
Cb6F14AGrvI has  comments disabled
OmpK2tkKmNQ has likes  disabled
SuApY0sEvz4 has  comments disabled
XJcM__haurU has  comments disabled
BMf9QEg3F-I has  comments disabled
LcyCs7fRmgc has  comments disabled
J10KDPg_Im0 has  comments disabled
7ghz_npkbmE has  comments disabled
JkGZ2Hl1l8c has  comments disabled
7CjAIbmZZGY has  comments disabled
bNMtxQnnuaA has  comments disabled
RlU32AfEVeU has likes  disabled
5037  videos to run semantic analysis on
{'https://en.wikipedia.org/wiki/Humour', 'https://en.wikipedia.org/wiki/Action_game', 'https://en.wikipedia.org/wiki/Video_game_culture', 'https://en.wikipedia.org/wiki/Music_of_Asia', 'https://en.wikipedia.org/wiki/Puzzle_video_game', 'https://en.wikipedia.org/wiki/Reggae', 'https://en.wikipedia.org/wiki/Casual_game', 'https://en.wikipedia.org/wiki/Independent_music', 'https://en.wikipedia.org/wiki/So

,kind,etag,items,pageInfo
0,youtube#videoListResponse,uihAK9XOb20ZMPAhyaOl3RlzxtM,"[{'kind': 'youtube#video', 'etag': 'gyYBtfDbC6...","{'totalResults': 50, 'resultsPerPage': 50}"
1,youtube#videoListResponse,3aVxIecKBk-9vNk69yhO9c-vvAw,"[{'kind': 'youtube#video', 'etag': 'xrTMrq3CmH...","{'totalResults': 50, 'resultsPerPage': 50}"
2,youtube#videoListResponse,oqZRkDSIFkHS_Dp55C-nLtyTLwc,"[{'kind': 'youtube#video', 'etag': 'lh7jNYD8ad...","{'totalResults': 50, 'resultsPerPage': 50}"
3,youtube#videoListResponse,tjecSLlBxgVHs0Azc6qKUd2P7AI,"[{'kind': 'youtube#video', 'etag': '0gDasbT46d...","{'totalResults': 50, 'resultsPerPage': 50}"
4,youtube#videoListResponse,UFeabLiGm4bJbeEeLhtsiSipWwk,"[{'kind': 'youtube#video', 'etag': 'jrRd2t92yo...","{'totalResults': 50, 'resultsPerPage': 50}"


In [17]:
#deepcopy
channelIdToListOfVids_copy = json.loads(json.dumps(channelIdToListOfVids))
debugMsg = []
for channelId in LIST_OF_CHANNEL_IDS:
    listOfVideosForEachChannel = channelIdToListOfVids_copy[channelId]
    for video in listOfVideosForEachChannel:
        additionalVideoMetadata= matching_vidId_to_info.get(video['vidId'])
        if additionalVideoMetadata:
            video.update(additionalVideoMetadata)
        else:
            debugMsg.append(video['vidId'])

print(sum(len(channelIdToListOfVids_copy[channelId]) for channelId in LIST_OF_CHANNEL_IDS))
print(len(debugMsg))
#some of these vids are private, duplicates. 

5224
187


In [18]:
REFERENCE_UTC_TIME_FOR_PUBLISH = datetime(2021, 12, 24, 5, 2, 59, 166993)
channelId_to_earliestVideoTimestamp_and_mostViewedVideo = {} # {channelId: {firstVideoTimestamp: <>, mostViewedVideoInfo: {}}}

#channelId_to_representative_vid_and_first_vid
for channelId in LIST_OF_CHANNEL_IDS:
    listOfVideoInfo = channelIdToListOfVids_copy[channelId]
    df = pd.DataFrame(list(listOfVideoInfo))
    mostViewedVideoIndex = df['viewCount'].idxmax()
    earliestVideoTimestamp = min(df['publishTime'])
    firstUploadDate = earliestVideoTimestamp
    daysLiveSinceFirstUpload = pd.Timedelta(REFERENCE_UTC_TIME_FOR_PUBLISH - datetime.strptime(earliestVideoTimestamp, '%Y-%m-%d'+'T'+'%H:%M:%S'+'Z')).round('d')
    channelId_to_earliestVideoTimestamp_and_mostViewedVideo[channelId] = {'firstUploadDate': firstUploadDate, 'daysSinceFirstVideo': daysLiveSinceFirstUpload, 'mostViewedVideo': df.iloc[mostViewedVideoIndex].to_dict()}

And now, I update the dictionary of each video specified by vidId, to update the video information with more metadata. Not only does the video info contain `vid publishedAt`, `publishTime`, `vidTitle`, and `vidDescription`, but it also contains `duration`, `viewCount`, `likeCount`, `commentCount`, and `topicCategories`

In [19]:
with open('masterSheetOfChannelIdToListOfVidInfos.json', 'w') as outfile:
    json.dump(channelIdToListOfVids_copy, outfile)

channelIdToListOfVidIds = {}
for channelId in LIST_OF_CHANNEL_IDS:
    vidInfoList = channelIdToListOfVids_copy[channelId]
    channelIdToListOfVidIds[channelId] = []
    for vidInfo in vidInfoList:
        channelIdToListOfVidIds[channelId].append(vidInfo['vidId'])

with open('masterSheetOfChannelIdToListOfVidIds.json', 'w') as outfile:
    json.dump(channelIdToListOfVidIds, outfile)

If I had a lot of resources like an actual GPU and memory, I'd probably analyze all the videos to find the average video to be representative of the channel, but since I don't, I want to take the most viewed video of each channel. This is a decision I made based on the... untestable hypothesis that the most-viewed video is the one that brought the channel the most subscriber count out of all of them. 

So I think taking the most-viewed video to represent the channel is an acceptable alternative. Thus, I will begin searching for 
1. the most viewed video by searching for `max(viewcount)` for each Channel
2. the first uploaded video by searching for `min(publishedAt)` for each channel because I need those stats as well. 

The result I get should be something along the lines of: `{ChannelId: 'xyz', earliestVideoPublishedDate: 'DateTime', mostViewedVideo : {vidInfo dictionary}}`

In [20]:
listOfVideoInfo = channelIdToListOfVids_copy["UC9z7EZAbkphEMg0SP7rw44A"]
df = pd.DataFrame(list(listOfVideoInfo)) 
df

,vidId,publishTime,vidTitle,duration,viewCount,likeCount,commentCount,topicCategories,description
0,42OleX0HR4E,2019-01-18T00:16:23Z,Cary teaches you how to time travel,PT2M45S,3398128.0,158540.0,20718.0,[https://en.wikipedia.org/wiki/Knowledge],"hello hello, in this educational video I will ..."
1,DxQQwExTjcM,2014-05-04T22:31:04Z,How strong is Steve? (Reuploaded again),PT2M12S,5303121.0,141953.0,29883.0,[https://en.wikipedia.org/wiki/Video_game_cult...,Original version (this video was first made in...
2,31dsH2Fs1IQ,2015-06-25T18:35:48Z,Evolution Simulator (Part 2/4),PT4M46S,2287609.0,28657.0,717.0,[https://en.wikipedia.org/wiki/Knowledge],Part 1: https://www.youtube.com/watch?v=GOFws_...
3,uaGEjrADGPA,2012-02-02T02:27:24Z,The Scale of the Universe 2,PT3M24S,2238626.0,17979.0,2140.0,[https://en.wikipedia.org/wiki/Knowledge],**Interactives**\nMobile Version:\nhttps://itu...
4,FlfOcIDRKPQ,2015-08-04T02:43:51Z,No More Lasers!,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
266,SKQJlvqhDzQ,2016-03-26T21:29:04Z,"GOLAD 3/26: undo/redo, symmetry",PT8M20S,8643.0,161.0,104.0,[https://en.wikipedia.org/wiki/Knowledge],VSauce — Michael here. Another dev vlog!\nⒼⓄⓁⒶ...
267,7SXr4u2L1yg,2011-12-17T22:18:23Z,My Minecraft Mod: Diamond Trees!,PT13M,7799.0,84.0,29.0,[https://en.wikipedia.org/wiki/Action-adventur...,The diamonds are finally harvested at 9:00\n\n...
268,6KrxVYo0mBQ,2012-03-05T05:27:14Z,How many comments?,PT17S,10349.0,77.0,759.0,[],This is an experiment. What will happen? Let...
269,mvzqt-BpS7w,2012-06-16T04:19:09Z,How do cats look out of windows?,PT4M19S,6119.0,97.0,41.0,[https://en.wikipedia.org/wiki/Lifestyle_(soci...,Here's how.


So... creating a dataframe out of the info is quite trivial. Really easy. I also just noticed that some of these youtube channels are going to have music videos... which is not in the scope of my project right now. Gonna have to remove any videos with topicCategories of music themes.

In [21]:
REFERENCE_UTC_TIME_FOR_PUBLISH = datetime(2021, 12, 24, 5, 2, 59, 166993)
channelId_to_earliestVideoTimestamp_and_mostViewedVideo = {} # {channelId: {firstVideoTimestamp: <>, mostViewedVideoInfo: {}}}

#channelId_to_representative_vid_and_first_vid
for channelId in LIST_OF_CHANNEL_IDS:
    listOfVideoInfo = channelIdToListOfVids_copy[channelId]
    df = pd.DataFrame(list(listOfVideoInfo))
    mostViewedVideoIndex = df['viewCount'].idxmax()
    earliestVideoTimestamp = min(df['publishTime'])
    firstUploadDate = earliestVideoTimestamp
    daysLiveSinceFirstUpload = pd.Timedelta(REFERENCE_UTC_TIME_FOR_PUBLISH - datetime.strptime(earliestVideoTimestamp, '%Y-%m-%d'+'T'+'%H:%M:%S'+'Z')).round('d')
    channelId_to_earliestVideoTimestamp_and_mostViewedVideo[channelId] = {'firstUploadDate': firstUploadDate, 'daysSinceFirstVideo': daysLiveSinceFirstUpload, 'mostViewedVideo': df.iloc[mostViewedVideoIndex].to_dict()}

In [22]:
def try_parsing_date(text):
    for fmt in ('%Y-%m-%d'+'T'+'%H:%M:%S'+'Z', '%Y-%m-%d'+'T'+'%H:%M.%S'+'Z'):
        try:
            return datetime.strptime(text, fmt)
        except ValueError:
            print(text)
            
    raise ValueError('no valid date format found')

In [23]:
#important. date ref is 12/11 because this is when I pulled this data. 

df = pd.read_json (r'channelListResponse.json')
REFERENCE_UTC_NOW= datetime(2021, 12, 24, 5, 2, 59, 166993)
channelInfo_master_df= pd.DataFrame()

for index, row in df.iterrows():
    items=df['items'][index][0]
    snippet = items['snippet']

    title=snippet['title']
    channelId=items['id']
    publishedAt=snippet['publishedAt']
    description=snippet['description']
    viewCount=items['statistics']['viewCount']
    subscriberCount=items['statistics']['subscriberCount']
    videoCount=items['statistics']['videoCount']
    daysLive = pd.Timedelta(REFERENCE_UTC_NOW - try_parsing_date(publishedAt)).round('d').days

    data = {'channelName':title, 
    'channelId': channelId, 
    'daysLiveAsOfLaunch': daysLive,
    'daysSinceFirstVideo': channelId_to_earliestVideoTimestamp_and_mostViewedVideo[channelId]['daysSinceFirstVideo'].days,
    'description': description,
    'viewCount': int(viewCount),
    'subscriberCount': int(subscriberCount),
    'videoCount': int(videoCount),
    'firstUploadDate' : channelId_to_earliestVideoTimestamp_and_mostViewedVideo[channelId]['firstUploadDate'],
    'subscribersPerYearSinceFirstUpload' : float(int(subscriberCount)/(channelId_to_earliestVideoTimestamp_and_mostViewedVideo[channelId]['daysSinceFirstVideo'].days/365)),
    'viewCountPerYearSinceFirstUpload' : float(int(viewCount)/(channelId_to_earliestVideoTimestamp_and_mostViewedVideo[channelId]['daysSinceFirstVideo'].days/365)),
    }
    df_generated = pd.DataFrame(data, index=[index])
    channelInfo_master_df= channelInfo_master_df.append(df_generated)
channelInfo_master_df.head()

2021-06-09T07:37.36Z


,channelName,channelId,daysLiveAsOfLaunch,daysSinceFirstVideo,description,viewCount,subscriberCount,videoCount,firstUploadDate,subscribersPerYearSinceFirstUpload,viewCountPerYearSinceFirstUpload
0,TJ Toons,UCnsem444vdU1HhS0mb2wwTA,3841,1276,Tellin my stories for fun,40741423,503000,15,2018-06-27T12:10:25Z,143883.228840,1.165409e+07
1,Alex's Corner,UCnbtlei4RJMHWSUq4LKn_SQ,1530,1528,Welcome to my corner~ \n\nThis is where I ramb...,27822141,648000,25,2017-10-18T07:43:58Z,154790.575916,6.645996e+06
2,RushLight Invader,UC9i7x4EzTBlNvslu1wWEB8Q,3311,3251,"Hello, I'm RushLight Invader! I make animation...",20078925,276000,46,2013-01-29T04:22:45Z,30987.388496,2.254324e+06
3,ChillyPanda,UC1nM46_BMKq4vd4jmd6iMHA,2075,2036,"Hey guys, it's Chilly~\n\nI honestly don't kno...",10232784,191000,51,2016-05-27T18:48:49Z,34241.159136,1.834463e+06
4,Recreyo,UCMNxwlfY6P5Rm40QOp8Hqsg,581,559,,42863332,340000,79,2020-06-12T19:00:07Z,222003.577818,2.798769e+07


In [24]:
with open("channelHolisticInfo.json", 'w') as f:
    json.dump(channelInfo_master_df.to_dict(), f)

`MostViewedVideoInfo` is showing up as NaN because it's of type `dict`. So need to just make sure I have access to the deets.

In [25]:
mostViewedVideo_master_dict = {}
for channelId in LIST_OF_CHANNEL_IDS:
    data = channelId_to_earliestVideoTimestamp_and_mostViewedVideo[channelId]['mostViewedVideo']
    mostViewedVideo_master_dict[channelId]= data
    
mostViewedVideo_master_df = pd.DataFrame.from_dict(mostViewedVideo_master_dict).T
mostViewedVideo_master_df

,vidId,publishTime,vidTitle,duration,viewCount,likeCount,commentCount,topicCategories,description
UCsDmESjqNPukDmVnuneLrqw,EcgkRp2IUsc,2017-10-26T01:05:22Z,My Hot Babysitter Got Me In Trouble,PT6M30S,35960829.0,840561.0,75550.0,"[https://en.wikipedia.org/wiki/Entertainment, ...",First Name Babysitter. Last Name Trouble. \nGE...
UCY6Ij8zOds0WJEeqCLOnqOQ,kbCah6yhYRs,2019-06-11T11:00:07Z,Descendants doesn&#39;t make any sense...,PT16M8S,9850298.0,263259.0,22459.0,"[https://en.wikipedia.org/wiki/Entertainment, ...",Descendants doesn't make any sense animation\n...
UClN24S5-fc7LD0JJW0FdGWw,ewsGmhAjjjI,2018-10-09T20:50:50Z,I have APHANTASIA (and you may too...without r...,PT12M54S,6041764.0,320113.0,72063.0,"[https://en.wikipedia.org/wiki/Hobby, https://...",So...yeah. I'm an artist that can't see pictur...
UC3iNdSV_RQU7DHTGpqEW96w,gA0bi-bFEYs,2018-06-22T19:20:36Z,How KSI Saved Logan Paul&#39;s Career,PT6M48S,20326487.0,514050.0,95645.0,"[https://en.wikipedia.org/wiki/Entertainment, ...",This KSI vs Logan Paul fight is interesting to...
UC_52i6mlrzOZs_NEuSepzAg,so1_5hYUEE8,2017-09-20T22:12:54Z,Being a Boba Barista (Work Stories),PT5M20S,2196765,58187,3756,[https://en.wikipedia.org/wiki/Lifestyle_(soci...,Get it? “B”eing a “B”oba “B”arista “b”y “B”ill...
...,...,...,...,...,...,...,...,...,...
UCnbtlei4RJMHWSUq4LKn_SQ,vv2vPAzj8S4,2018-12-14T21:00:03Z,How I Met My &quot;Boyfriend&quot; (ft. Sultan...,PT8M49S,5999839,221481,7793,"[https://en.wikipedia.org/wiki/Entertainment, ...",Part 1 out of 3 of my Rebound Relationship Ser...
UC9i7x4EzTBlNvslu1wWEB8Q,Xnv7JGqjaAo,2018-07-19T17:21:51Z,Life is Fun - TheOdd1sOut - Animation Breakdown,PT5M17S,2283034,43407,2887,"[https://en.wikipedia.org/wiki/Entertainment, ...",I Animated the Chorus for TheOdd1sOut Life is ...
UC1nM46_BMKq4vd4jmd6iMHA,IwxWmKsVR5U,2017-06-05T22:09:49Z,Private School,PT6M23S,850879,16746,2733,[],Who would've thought I went to a private catho...
UCMNxwlfY6P5Rm40QOp8Hqsg,2bGkEK8I6zQ,2021-03-19T19:30:00Z,Can They Survive My Hero Academia?,PT18M26S,2482107,88206,3657,"[https://en.wikipedia.org/wiki/Entertainment, ...",#Animation #MyHeroAcademia #Season5\n\nVisit h...


So I need to remove... all the music-themed game letsplay stuff. And I did. All the way up there.

Whew. And now... I need to gather the transcripts for the 33 videos above to run semantic analysis on it. There's probably an API for it...

In [26]:
channelId_to_mostViewedVidId = {}

for channelId in LIST_OF_CHANNEL_IDS:
    mostViewedVidId = mostViewedVideo_master_dict[channelId]['vidId']
    channelId_to_mostViewedVidId[channelId] = mostViewedVidId

with open('channelId_to_mostViewedVidId.json', 'w') as outfile:
    json.dump(channelId_to_mostViewedVidId, outfile)